# Imports

In [1]:
%matplotlib tk

from tkinter import *
from PIL import ImageTk, Image
import itertools
import serial
import time
import pandas as pd
import numpy as np
import sys
import glob
import serial
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.animation as animation
from tkinter import filedialog
import os
from datetime import datetime

# Creating a CSV file from sensor inputs

In [2]:
# def create_CSV():
#     PORTS = serial_ports()
#     for port_name in PORTS:
#         OUTPUTS =[]
#         if(port_name[9] == 'u'):
#             ser = serial.Serial(port = port_name, baudrate = 9600, timeout = 0)
#             with serial.Serial(port_name, baudrate=9600) as ser:
#                 for i in range(100):
#                     s = ser.read(4)        # read up to ten bytes (timeout)
#                     OUTPUTS.append(int.from_bytes(s[:-1], "little"))
#         df = pd.DataFrame(OUTPUTS, columns=[port_name])
#     df.to_csv('~/Desktop/oqni/PORTS.csv', index=False)
#     info_clicked("Succesfully Created the CSV file", info_frame)

In [3]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [4]:
def is_clicked(record_protocol, test_window):
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+ record_protocol.replace('\\','')+'.csv')
    myLabel = Label(test_window, text = '000000', bg="white", fg="white")
    myLabel.config(font=('Helvetica bold', 26))
    myLabel.grid(row=4,column=1)
    myLabel = Label(test_window, text = exp_data['time_interval'].sum(), bg="white")
    myLabel.config(font=('Helvetica bold', 26))
    myLabel.grid(row=4,column=1)
    
def testing():
    test_window = Toplevel()
    test_window.title("Test Run")
    
    default_path = '/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/recordings/'
    protocols = [] #for stering Exp names

    #Getting protocols from folder and storing them
    with os.scandir('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols') as entries:
        for entry in entries:
            if(entry.name[0] == 'E'):
                protocols.append((entry.name)[:-4])
    protocols.sort()

    # setting the windows size
    test_window.geometry("850x200+0+480")

    # declaring string variable for storing Entries
    database_path= StringVar(test_window)
    recording_folder= StringVar(test_window)
    subject_number= StringVar(test_window)
    date_yymmdd= StringVar(test_window)
    time_s= StringVar(test_window)
    OPT_placement = StringVar(test_window)
    IMU_placement = StringVar(test_window)
    record_protocol = StringVar(test_window)
    limb = StringVar(test_window)

    #setting entries
    database_path.set(default_path)
    recording_folder.set("BL-001")
    subject_number.set("000")
    date_yymmdd.set("000000")
    time_s.set("1")
    OPT_placement.set("1.1, 1.2")
    opt_dropdown = OptionMenu(test_window, OPT_placement, "1.1, 1.2", "2.1, 2.2", "3.1, 3.2")
    IMU_placement.set("above knee")
    imu_dropdown = OptionMenu(test_window, IMU_placement, "above knee", "below knee")
    record_protocol.set("Exp\#1")
    protocol_dropdown = OptionMenu(test_window, record_protocol, *protocols, command=lambda record_protocol: is_clicked(record_protocol, test_window))
    limb.set("left leg")
    limb_dropdown = OptionMenu(test_window, limb, "left leg", "right leg")

    #Creating labels and entries
    database_label = Label(test_window, text = 'Database Path', font=('calibre',10, 'bold'))
    database_entry = Entry(test_window,width=90, textvariable = database_path, font=('calibre',10,'normal'))
    recording_label = Label(test_window, text = 'Recording folder', font = ('calibre',10,'bold'))
    recording_entry=Entry(test_window, textvariable = recording_folder, font = ('calibre',10,'normal'))
    subject_entry = Entry(test_window, textvariable = subject_number, font = ('calibre',10,'bold'))
    date_entry = Entry(test_window, textvariable = date_yymmdd, font = ('calibre',10,'bold'))
    time_entry = Entry(test_window, textvariable = time_s, font = ('calibre',10,'bold'))
    OPT_label = Label(test_window, text = 'OPT Placement:', font=('calibre',10, 'bold'))
    IMU_label = Label(test_window, text = "IMU Placement:", font=('calibre',10, 'bold'))
    protocol_label = Label(test_window,text = "Recording Protocol:", font=('calibre',10, 'bold'))
    limb_label = Label(test_window,text = "Limb:", font=('calibre',10, 'bold'))

    #Label for showing overall seconds of a specific protocol
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+ (record_protocol.get()).replace('\\','')+'.csv')
    time_interval = Label(test_window, text = exp_data['time_interval'].sum())
    time_interval.config(font=('Helvetica bold', 26))
    protocol_length_label = Label(test_window, text = "Protocol Length:", font=('calibre',10, 'bold'))

    #Creating buttons
    start_button=Button(test_window,text = 'Start', command = lambda: start(database_path,recording_folder,subject_number,date_yymmdd,time_s,OPT_placement,IMU_placement,record_protocol,limb,test_window))
    browsebutton = Button(test_window, text="Browse", command=lambda: browsefunc(database_path))
    stop_button = Button(test_window, text="Stop", command = stop)

    root.update() #Did not update entries otherwise

    #Putting everything on a grid
    database_label.grid(row=0,column=0)
    database_entry.grid(row=0, column=1, columnspan=4)
    browsebutton.grid(row=0,column=5)
    recording_label.grid(row=1,column=0)
    recording_entry.grid(row=1,column=1)
    subject_entry.grid(row=1,column=2)
    date_entry.grid(row=1,column=3)
    time_entry.grid(row=1,column=4)
    OPT_label.grid(row=2, column=0)
    opt_dropdown.grid(row=2, column=1)
    IMU_label.grid(row=2,column=2)
    imu_dropdown.grid(row=2,column=3)
    protocol_label.grid(row=3,column=0)
    protocol_dropdown.grid(row=3,column=1)
    limb_label.grid(row=3, column=2)
    limb_dropdown.grid(row=3, column=3)
    start_button.grid(row=3,column=5)
    protocol_length_label.grid(row=4,column=0)
    time_interval.grid(row=4,column=1)
    stop_button.grid(row=4,column=5)

    test_window.mainloop()

In [5]:
def info_clicked_2(value, root):
    myLabel = Label(root, text = '000000000', bg="white", fg="white")
    myLabel.config(font=('Helvetica bold', 26))
    myLabel.grid(row=4,column=2)
    myLabel = Label(root, text = value, bg="white")
    myLabel.config(font=('Helvetica bold', 26))
    myLabel.grid(row=4,column=2)
def stop():
    global stopped
    stopped = 1
def browsefunc(database_path):
    filename = filedialog.askdirectory()
    database_path.set(filename)

In [6]:
def start(database_path,recording_folder,subject_number,date_yymmdd,time_s,OPT_placement,IMU_placement,record_protocol,limb, test_window):
    data_top = Toplevel()
    data_top.geometry('600x800+700+0')
    data_top.title("Test Run")
    
    #This variable is used for stopping the testing process
    global stopped
    stopped = 0
    
    #As soon as the start is clicked, the dates are being set
    date_yymmdd.set(datetime.today().strftime('%Y-%m-%d'))
    time_s.set(datetime.today().strftime('%H:%M:%S'))
    
    #Getting the user inputs
    path = database_path.get()
    folder = recording_folder.get()
    subject = subject_number.get()
    date = date_yymmdd.get()
    my_time = time_s.get()
    opt = OPT_placement.get()
    imu = IMU_placement.get()
    protocol = record_protocol.get()
    lim = limb.get()
    
    #getting the data of specific protocol
    exp_data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/'+protocol.replace('\\','')+'.csv')
    overall_time = exp_data['time_interval'].sum()
    
    for i in range(len(exp_data)): #Looping through images
        frame = Frame(data_top, width=600, height=400)
        frame.pack()
        frame.place(anchor='center', relx=0.5, rely=0.5)
        leg_image = PhotoImage(file='/Users/armenmkrtumyan/Desktop/oqni/DataLogger_STM/protocols/pics/'+str(exp_data['label_index'][i]) +'.png')
        my_label = Label(frame, image=leg_image)
        my_label['image'] = leg_image
        my_label.pack()
        k = exp_data['time_interval'][i]
        while(k > 0 and stopped != 1):
            info_clicked_2(str(overall_time) + ":" + str(k) + " " + str(exp_data['labels'][i]), test_window)
            data_top.update()
            time.sleep(1)
            k-=1
            overall_time-=1
            root.update()  
        data_top.update()
        for widget in frame.winfo_children():
            widget.destroy()
        if(stopped == 1):
            break
    os.chdir(str(path))
    if ((os.path.exists(str(folder)+'/')) and (stopped != 1)):
        os.chdir(str(folder)+'/')
        exp_data.to_csv(os.getcwd()+ "/" + str(date) + "-" + str(my_time).replace(':', '-') + '.csv')
    else:
        if(stopped != 1):
            os.mkdir(str(folder)+'/')
        else:
            if(not os.path.exists('BL-000/')):
                os.mkdir('BL-000/')
            folder = 'BL-000'
        os.chdir(str(folder)+'/')
        exp_data.to_csv(os.getcwd()+ "/" + str(date) + "-" + str(my_time).replace(':', '-') + '.csv')
    info_clicked_2('', test_window)
    data_top.destroy()
    stopped = 0

### Live plotting

In [7]:
def open_graph(port_name):
    global ani

    # Parameters
    x_len = 200         # Number of points to display
    y_range = [70, 80]  # Range of possible Y values to display

    # Create figure for plotting
    fig = plt.figure(figsize=(9.3,6.6), dpi=150)
    fig.canvas.manager.set_window_title('Sesnor inputs')

    ax = fig.add_subplot(1, 1, 1)
    xs = list(range(0, 200))
    ys = [0] * x_len
    ax.set_ylim(y_range)

    # Create a blank line. We will update the line in animate
    line, = ax.plot(xs, ys)

    # Add labels
    plt.title('Sensor Inputs over time')
    plt.xlabel('Time')
    plt.ylabel('Input')

    # This function is called periodically from FuncAnimation
    def animate(i, ys):
        temp_c = np.random.randint(70,80)
        # Add y to list
        ys.append(temp_c)
        # Limit y list to set number of items
        ys = ys[-x_len:]
        # Update line with new Y values
        line.set_ydata(ys)
        return line,

    # Set up plot to call animate() function periodically
    ani = animation.FuncAnimation(fig,
        animate,
        fargs=(ys,),
        interval=50,
        blit=True)
    plt.show()


In [8]:
def check_states(checkbox_states):
    for i, var in enumerate(checkbox_states):
        if var.get() == 1:
            print(f"Checkbox {i+1} is checked")
        else:
            print(f"Checkbox {i+1} is unchecked")

In [ ]:
root = Tk()
root.title("Port Reader")
root.geometry('700x400+0+0')
root.minsize(700, 400)
root.maxsize(700, 400)

bg = PhotoImage(file='/Users/armenmkrtumyan/Downloads/abstract-science-background-molecule-abstract-science-background-molecule-cells-beautiful-glow-10081.png')
my_label = Label(root, image=bg)
my_label.place(x=0,y=0, relwidth=1, relheight=1)

port_frame = LabelFrame(root, text="PORTS", padx = 0, pady = 0)
port_frame.place(x=10,y=10,height=380,width=300)
actions_frame = LabelFrame(root, text="ACTIONS", padx = 50, pady = 50)
actions_frame.place(x=320, y = 10, height = 160, width = 200)

PORTS = serial_ports()

checkbox_states = []
for i in range(len(PORTS)):
    var = IntVar()
    checkbox = Checkbutton(port_frame, text=PORTS[i], variable=var)
    checkbox.pack()
    checkbox_states.append(var)    

    
port_variable = StringVar()

root.update()

graph_button = Button(actions_frame, text = "Open graph", command = lambda: open_graph(port_variable.get()))
graph_button.place(in_ = actions_frame, x = -40, y= -40)

testing_button = Button(actions_frame, text = "Testing", command = lambda: testing())
testing_button.place(in_ = actions_frame, x = -40, y = 50)

# Create a button to check the state of the checkboxes
check_button = Button(port_frame, text="Choose Ports", command=lambda:check_states(checkbox_states))
check_button.pack()

mainloop()